In [38]:
import numpy as np
import pandas as pd
import os
from matplotlib import animation
from matplotlib import pyplot as plt
from matplotlib import patches
from matplotlib import cm
import matplotlib as mpl
from scipy.signal import butter, filtfilt
from scipy.interpolate import interp1d
from Tomato3_dataStructure import Trial, Subject, Experiment
from helper import angle_overtake, lateral_overtake


In [41]:
'''import trials'''
Hz = 90
exp = Experiment()
for i in range(1,13):
    exp.subjects[i] = Subject(i)
    
input_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Tomato3_rawData\Tomato3_input'))
output_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Tomato3_rawData\Tomato3_output'))

for output_file in os.listdir(output_dir):
    output_file_path = os.path.join(output_dir, output_file)
    # import experimental data
    if  'Tomato3_subj' in output_file and '.csv' in output_file:
        with open(output_file_path, 'r') as f:
            df = pd.read_csv(f, header=None)
            lpos = np.array(df.iloc[:, [0,2,1]])
            fpos = np.array(df.iloc[:, [3,5,4]])
            fori = np.array(df.iloc[:, 6:9])
            tstamps = np.array(df.iloc[:, 9])
            leader_model = np.array(df.iloc[:, 10])
            subject_id = int(output_file[12:14])
            trial_id = int(output_file[20:23])
            v0 = float(output_file[27:30])
            if output_file[-5] == 'e':
                leader = 'pole'
            elif output_file[-5] == 'r':
                leader = 'avatar'
            leader_onset = None
            
            exp.subjects[subject_id].trials[trial_id] = Trial(subject_id, trial_id, lpos, fpos, fori, \
                                                                tstamps, v0, leader, leader_onset, leader_model)           
    # import IPD and gender data
    elif 'IPD' in output_file:    
        with open(output_file_path, 'r') as f:
            subject_id = int(output_file[12:14])
            exp.subjects[subject_id].gender == output_file[19]
            i = output_file.find('txt')
            exp.subjects[subject_id].IPD == float(output_file[20:i-1])
    # import freewalk data
    elif 'freewalk' in output_file:
        with open(output_file_path, 'r') as f:
            df = pd.read_csv(f, header=None)
            subject_id = int(output_file[21:23])
            session = int(output_file[-14])
            trial_id = int(output_file[-7:-4])
            v0 = 0
            leader = leader_onset = leader_model= None
            fpos = np.array(df.iloc[:,[0,2,1]])
            lpos = np.tile([0,0,0], (len(fpos), 1))
            fori = np.array(df.iloc[:,3:6])
            tstamps = np.array(df.iloc[:,-1])
            if session == 1:
                exp.subjects[subject_id].freewalk[trial_id] = Trial(subject_id, trial_id, lpos, fpos, fori, \
                                                                    tstamps, v0, leader, leader_onset, leader_model)
            else:
                trial_id += 4
                exp.subjects[subject_id].freewalk[trial_id] = Trial(subject_id, trial_id, lpos, fpos, fori, \
                                                                    tstamps, v0, leader, leader_onset, leader_model)
# import inputs
for input_file in os.listdir(input_dir):
    if 'Tomato3_subject' in input_file:
        subject_id = int(input_file[-6:-4])
        if subject_id in exp.subjects and exp.subjects[subject_id].trials != {}:
            # read experimental trials
            with open(os.path.join(input_dir, input_file), 'r') as f:
                df = pd.read_csv(f)
                for i in range(len(df)):
                    trial_id = df.iloc[i,0]
                    leader_onset = df.iloc[i,4]
                    exp.subjects[subject_id].trials[trial_id].leader_onset = leader_onset
 

# with open(filename, 'r') as file:
#     rows = file.read().split('\n')[:-1]
#     cells = [row.split(',') for row in rows]  
#     cells = np.array([[float(s) for s in row] for row in cells])
#     lpos = cells[:,[0,2,1]]
#     fpos = cells[:,[3,5,4]]
#     fori = cells[:,6:9]
#     tstamps = cells[:,-1]



In [29]:
'''debug'''
%matplotlib qt
s = 20
t = 31
# print('subject_id ', exp.subjects[s].trials[t].subject_id)
# print('trial_id ', exp.subjects[s].trials[t].trial_id)
# print('d0 ', exp.subjects[s].trials[t].d0)
# print('v0 ', exp.subjects[s].trials[t].v0)
# print('tstamps ', exp.subjects[s].trials[t].tstamps)
# print('leader ', exp.subjects[s].trials[t].leader)
# print('leader_onset ', exp.subjects[s].trials[t].leader_onset)
# print('leader_model ', exp.subjects[s].trials[t].leader_model)
# print('order ', exp.subjects[s].trials[t].order)
# print('cutoff ', exp.subjects[s].trials[t].cutoff)

# # check d0
# trials = range(1, 61)
# for i in trials:
#     f1 = exp.subjects[s].trials[i].f1
#     fpos = exp.subjects[s].trials[i].fpos[f1]
#     lpos = exp.subjects[s].trials[i].lpos[f1]
#     distance = np.sqrt((lpos[0] - fpos[0])**2 + (lpos[1] - fpos[1])**2)
#     lpos_filt = exp.subjects[s].trials[i].get_positions('l')[f1]
#     fpos_filt = exp.subjects[s].trials[i].get_positions('f')[f1]
#     distance_filt = np.sqrt((lpos_filt[0] - fpos_filt[0])**2 + (lpos_filt[1] - fpos_filt[1])**2)
#     print('raw ', distance, ' filtered ', distance_filt)

# f1 = exp.subjects[s].trials[i].f1
# t = exp.subjects[s].trials[i].tstamps
# raw = exp.subjects[s].trials[i].get_positions('f', is_filtered=False)



In [3]:
'''plot positions or speed of experimental trials'''
%matplotlib qt
subject = 12
trials = list(range(1,61))
for i in trials:
    if exp.subjects[subject].trials[i].v0 == 0.8:
        exp.subjects[subject].trials[i].plot_positions(accelerations=False, links=True)
        
#         exp.subjects[subject].trials[i].plot_positions(accelerations=False, links=False, is_filtered=False)
#         exp.subjects[subject].trials[i].plot_speeds()

In [34]:
'''Plot positions or speed of freewalk trials'''
subject = 9
for i, t in exp.subjects[subject].freewalk.items():
    t.plot_positions()


In [20]:
'''animation'''
# compare trial 6 (catch up) and 5 (let it go)
%matplotlib qt
subject = 1
trial = 49
exp.subjects[subject].trials[trial].play_trial(save=False, is_filtered=True)

In [36]:
'''
    Percent overtaking by leader speed by subject (12 plots). Overtaking is defined by the angle y axis and the line
    connecting follower and leader greater than 55 degree. 55 is half of field of view in Odyssey HMD.
'''
%matplotlib qt
for i, s in exp.subjects.items():
    ot_angle = {0.8:0, 0.9:0, 1.0:0, 1.1:0, 1.2:0, 1.3:0}
    ot_lateral = {0.8:0, 0.9:0, 1.0:0, 1.1:0, 1.2:0, 1.3:0}
    for j, t in s.trials.items():
        lpos = t.get_positions('l')
        fpos = t.get_positions('f')      
        if angle_overtake(lpos, fpos, 55):
            ot_angle[t.v0] += 0.1
        if lateral_overtake(fpos, 0.4):
            ot_lateral[t.v0] += 0.1
    fig = plt.figure()
    ax = plt.axes(xlim=(0.7, 1.4), ylim=(-0.1, 1.1))
    ax.plot([x/10.0 for x in range(8, 14)],list(ot_angle.values()))
    ax.plot([x/10.0 for x in range(8, 14)],list(ot_lateral.values()))

In [7]:
'''
    Percent overtaking by speed difference by subject (12 plots). Overtaking is defined by the angle y axis and the line
    connecting follower and leader greater than 55 degree. 55 is half of field of view in Odyssey HMD.
'''
threshold = np.cos(55 * np.pi / 180)
for i, s in exp.subjects.items():
    dvs = []
    for j, t in s.trials.items():
        lpos = t.get_positions('l')[-1, 0:2]
        fpos = t.get_positions('f')[-1, 0:2]
        vec1 = lpos - fpos
        vec0 = [0, 1]
        cos = np.dot(vec0, vec1)/np.linalg.norm(vec1)        
        if cos <= threshold:
            fspd = t.get_speeds('f')[t.f1]
            dvs.append(fspd-t.v0)
    fig = plt.figure()
    ax = plt.axes()
    ax.hist(dvs, 20)

In [ ]:
'''Percent overtaking by leader (2 plots)'''


In [27]:
'''lateral deviation in freewalk trials'''
max_deviations = {}
for i, s in exp.subjects.items():
    max_deviation = 0
    for j, t in s.freewalk.items():
        _max = max(t.get_positions('f')[:, 0])
        if _max > max_deviation:
            max_deviation = _max
    max_deviations[i] = round(max_deviation, 3)
print(max_deviations)

{1: 0.088, 2: 0.097, 3: 0.207, 4: 0.205, 5: 0.235, 6: 0.144, 7: 0.072, 8: 0.148, 9: 0.394, 10: 0.132, 11: 0.128, 12: 0.082}
